# 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 2. Features to be Used

Below are the preliminary features that will be used to build the model. More will be added on afterwards.

**Fatigue**
- Days since last match

**Home Team Form**
- Goals difference of home team in the last x matches    
- Goals difference of home team in the last x home matches    
- Average number of points gained by home team in the last x matches
- Average number of points gained by home team in the last x home matches
- Home Team Win streak 
- Number of home matches won by home team in its last x home matches?
- Home Team Newly Promoted Team?

**Away Team Form**
- Goals difference of away team in the last x matches  
- Goals difference of away team in the last x away matches
- Average number of points gained by away team in the last x matches
- Average number of points gained by away team in the last x away matches
- Away Team Win streak
- Number of away matches won by away team in its last x away matches?
- Away Team Newly Promoted Team?

**Home Team Performance Index**
- Home Defense Performance Index
- Home Midfield Performance Index
- Home Attack Performance Index

**Away Team Performance Index**
- Away Defense Performance Index
- Away Midfield Performance Index
- Away Attack Performance Index

**Betting Odds**
- B365H
- B365D
- B365A

In [3]:
df = pd.read_csv('datasets/dataset.csv')
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HxG,AxG,HPoss,HDaysLastPlayed,ADaysLastPlayed
0,2017-08-11,arsenal,leicester,4,3,27,6,10,3,9,12,9,4,0,1,0,0,1.53,4.5,6.50,W,2.5,1.5,68.0,5,10
1,2017-08-12,brighton,city,0,2,6,14,2,4,6,9,3,10,0,2,0,0,11.00,5.5,1.33,L,0.3,1.9,23.0,10,10
2,2017-08-12,chelsea,burnley,2,3,19,10,6,5,16,11,8,5,3,3,2,0,1.25,6.5,15.00,L,1.5,0.6,62.0,6,10
3,2017-08-12,palace,huddersfield,0,3,14,8,4,6,7,19,12,9,1,3,0,0,1.83,3.6,5.00,L,1.1,1.5,56.0,10,10
4,2017-08-12,everton,stoke,1,0,9,9,4,1,13,10,6,7,1,1,0,0,1.70,3.8,5.75,W,0.6,0.4,60.0,9,10


In [4]:
df.shape

(1900, 26)

# 3. EDA

# 4. Feature Engineering

## Goals difference of home team in the last x matches

In [5]:
def getGoalsScoredByTeam(row, team):
    if row['HomeTeam'] == team:
        return row['FTHG']
    elif row['AwayTeam'] == team:
        return row['FTAG']

In [6]:
def getGoalsScoredByOpponent(row, team):
    if row['HomeTeam'] == team:
        return row['FTAG']
    elif row['AwayTeam'] == team:
        return row['FTHG']

In [7]:
temp_df = pd.DataFrame()

for team in df['HomeTeam'].unique():
    team_df = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)]
    team_df = team_df.sort_values(by = 'Date')
    
    number_of_previous_matches = 5
    column1 = 0;
    column2 = 0;
    
    team_df['GoalsScoredByTeam'] = team_df.apply(lambda row: getGoalsScoredByTeam(row, team), axis=1)
    team_df['GoalsScoredByOpponent'] = team_df.apply(lambda row: getGoalsScoredByOpponent(row, team), axis=1)
    
    for i in range(1, number_of_previous_matches+1):
        column1 += team_df['GoalsScoredByTeam'].shift(i)
        column2 += team_df['GoalsScoredByOpponent'].shift(i)
    
    team_df['HxGD in Last 5 Matches'] = column1 - column2
    team_df.drop(columns=['GoalsScoredByTeam', 'GoalsScoredByOpponent'], inplace=True)
    team_df = team_df[team_df['HomeTeam'] == team]
    temp_df = pd.concat([temp_df, team_df]).reset_index(drop=True)

df = temp_df

In [8]:
df.head(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HxG,AxG,HPoss,HDaysLastPlayed,ADaysLastPlayed,HxGD in Last 5 Matches
0,2017-08-11,arsenal,leicester,4,3,27,6,10,3,9,12,9,4,0,1,0,0,1.53,4.50,6.50,W,2.5,1.5,68.0,5,10,NaN
1,2017-09-09,arsenal,bournemouth,3,0,17,7,9,2,14,10,10,3,0,1,0,0,1.36,5.50,9.00,W,2.2,0.6,58.0,10,10,NaN
2,2017-09-25,arsenal,west-brom,2,0,16,7,6,3,8,17,7,4,1,4,0,0,1.33,5.50,11.00,W,2.2,0.9,69.0,5,5,-1.0
3,2017-10-01,arsenal,brighton,2,0,25,9,12,1,7,8,6,5,0,2,0,0,1.20,6.50,13.00,W,2.4,0.4,64.0,3,7,0.0
4,2017-10-28,arsenal,swansea,2,1,17,4,5,2,9,9,5,2,0,0,0,0,1.25,7.00,12.00,W,2.0,0.9,72.0,4,4,6.0
5,2017-11-18,arsenal,tottenham,2,0,14,14,5,4,11,16,7,4,4,1,0,0,2.50,3.60,2.89,W,2.1,0.7,43.0,10,10,3.0
6,2017-11-29,arsenal,huddersfield,5,0,21,7,7,2,10,12,7,2,0,1,0,0,1.19,7.50,17.00,W,4.0,0.5,69.0,3,3,5.0
7,2017-12-02,arsenal,united,1,3,33,8,15,4,11,10,12,1,3,2,0,1,2.50,3.39,3.00,L,4.7,1.8,75.0,3,4,7.0
8,2017-12-16,arsenal,newcastle,1,0,23,10,5,2,13,9,7,5,2,1,0,0,1.25,6.50,14.00,W,1.8,0.3,71.0,3,3,4.0
9,2017-12-22,arsenal,liverpool,3,3,11,14,4,9,9,11,7,8,1,0,0,0,2.54,3.60,2.79,D,1.2,2.0,54.0,3,5,4.0


In [9]:
df.shape

(1900, 27)

## Goals difference of home team in the last x home matches

In [10]:
temp_df = pd.DataFrame()

for team in df['HomeTeam'].unique():
    team_df = df[df['HomeTeam'] == team]
    
    number_of_previous_matches = 5
    column1 = 0;
    column2 = 0
    
    team_df = team_df.sort_values(by = 'Date')
    for i in range(1, number_of_previous_matches+1):
        column1 += team_df['FTHG'].shift(i)
        column2 += team_df['FTAG'].shift(i)

    team_df['HxGD in Last 5 Home Matches'] = column1 - column2
    temp_df = pd.concat([temp_df, team_df]).reset_index(drop=True)

df = temp_df

In [11]:
df.head(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HxG,AxG,HPoss,HDaysLastPlayed,ADaysLastPlayed,HxGD in Last 5 Matches,HxGD in Last 5 Home Matches
0,2017-08-11,arsenal,leicester,4,3,27,6,10,3,9,12,9,4,0,1,0,0,1.53,4.50,6.50,W,2.5,1.5,68.0,5,10,NaN,NaN
1,2017-09-09,arsenal,bournemouth,3,0,17,7,9,2,14,10,10,3,0,1,0,0,1.36,5.50,9.00,W,2.2,0.6,58.0,10,10,NaN,NaN
2,2017-09-25,arsenal,west-brom,2,0,16,7,6,3,8,17,7,4,1,4,0,0,1.33,5.50,11.00,W,2.2,0.9,69.0,5,5,-1.0,NaN
3,2017-10-01,arsenal,brighton,2,0,25,9,12,1,7,8,6,5,0,2,0,0,1.20,6.50,13.00,W,2.4,0.4,64.0,3,7,0.0,NaN
4,2017-10-28,arsenal,swansea,2,1,17,4,5,2,9,9,5,2,0,0,0,0,1.25,7.00,12.00,W,2.0,0.9,72.0,4,4,6.0,NaN
5,2017-11-18,arsenal,tottenham,2,0,14,14,5,4,11,16,7,4,4,1,0,0,2.50,3.60,2.89,W,2.1,0.7,43.0,10,10,3.0,9.0
6,2017-11-29,arsenal,huddersfield,5,0,21,7,7,2,10,12,7,2,0,1,0,0,1.19,7.50,17.00,W,4.0,0.5,69.0,3,3,5.0,10.0
7,2017-12-02,arsenal,united,1,3,33,8,15,4,11,10,12,1,3,2,0,1,2.50,3.39,3.00,L,4.7,1.8,75.0,3,4,7.0,12.0
8,2017-12-16,arsenal,newcastle,1,0,23,10,5,2,13,9,7,5,2,1,0,0,1.25,6.50,14.00,W,1.8,0.3,71.0,3,3,4.0,8.0
9,2017-12-22,arsenal,liverpool,3,3,11,14,4,9,9,11,7,8,1,0,0,0,2.54,3.60,2.79,D,1.2,2.0,54.0,3,5,4.0,7.0


In [12]:
df.shape

(1900, 28)

## Average number of points gained by home team in the last x matches

In [13]:
def getPointsGained(row):
    if row['Result'] == 'W':
        return 3
    elif row['Result'] == 'D':
        return 1
    elif row['Result'] == 'L':
        return 0

In [14]:
temp_df = pd.DataFrame()

for team in df['HomeTeam'].unique():
    team_df = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)]
    team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row), axis=1)
    
    number_of_previous_matches = 5
    column = 0
    
    team_df = team_df.sort_values(by = 'Date')
    for i in range(1, number_of_previous_matches+1):
        column += team_df['Points Gained'].shift(i)
    
    team_df['HAvgPts in Last 5 Matches'] = column / number_of_previous_matches
    team_df.drop(columns=['Points Gained'], inplace=True)
    
    team_df = team_df[team_df['HomeTeam'] == team]
    temp_df = pd.concat([temp_df, team_df]).reset_index(drop=True)

df = temp_df

C:\Users\choibs\AppData\Local\Temp\ipykernel_10148\2024999220.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row), axis=1)
C:\Users\choibs\AppData\Local\Temp\ipykernel_10148\2024999220.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row), axis=1)
C:\Users\choibs\AppData\Local\Temp\ipykernel_10148\2024999220.py:5: SettingWithCopyWarning: 
A value is trying to be s

In [15]:
df.head(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HxG,AxG,HPoss,HDaysLastPlayed,ADaysLastPlayed,HxGD in Last 5 Matches,HxGD in Last 5 Home Matches,HAvgPts in Last 5 Matches
0,2017-08-11,arsenal,leicester,4,3,27,6,10,3,9,12,9,4,0,1,0,0,1.53,4.50,6.50,W,2.5,1.5,68.0,5,10,NaN,NaN,NaN
1,2017-09-09,arsenal,bournemouth,3,0,17,7,9,2,14,10,10,3,0,1,0,0,1.36,5.50,9.00,W,2.2,0.6,58.0,10,10,NaN,NaN,NaN
2,2017-09-25,arsenal,west-brom,2,0,16,7,6,3,8,17,7,4,1,4,0,0,1.33,5.50,11.00,W,2.2,0.9,69.0,5,5,-1.0,NaN,2.6
3,2017-10-01,arsenal,brighton,2,0,25,9,12,1,7,8,6,5,0,2,0,0,1.20,6.50,13.00,W,2.4,0.4,64.0,3,7,0.0,NaN,2.6
4,2017-10-28,arsenal,swansea,2,1,17,4,5,2,9,9,5,2,0,0,0,0,1.25,7.00,12.00,W,2.0,0.9,72.0,4,4,6.0,NaN,2.0
5,2017-11-18,arsenal,tottenham,2,0,14,14,5,4,11,16,7,4,4,1,0,0,2.50,3.60,2.89,W,2.1,0.7,43.0,10,10,3.0,9.0,2.4
6,2017-11-29,arsenal,huddersfield,5,0,21,7,7,2,10,12,7,2,0,1,0,0,1.19,7.50,17.00,W,4.0,0.5,69.0,3,3,5.0,10.0,1.8
7,2017-12-02,arsenal,united,1,3,33,8,15,4,11,10,12,1,3,2,0,1,2.50,3.39,3.00,L,4.7,1.8,75.0,3,4,7.0,12.0,2.4
8,2017-12-16,arsenal,newcastle,1,0,23,10,5,2,13,9,7,5,2,1,0,0,1.25,6.50,14.00,W,1.8,0.3,71.0,3,3,4.0,8.0,1.0
9,2017-12-22,arsenal,liverpool,3,3,11,14,4,9,9,11,7,8,1,0,0,0,2.54,3.60,2.79,D,1.2,2.0,54.0,3,5,4.0,7.0,1.6


In [16]:
df.shape

(1900, 29)

## Average number of points gained by home team in the last x home matches

In [17]:
temp_df = pd.DataFrame()

for team in df['HomeTeam'].unique():
    team_df = df[(df['HomeTeam'] == team)]
    team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row), axis=1)
    
    number_of_previous_matches = 5
    column = 0
    
    team_df = team_df.sort_values(by = 'Date')
    for i in range(1, number_of_previous_matches+1):
        column += team_df['Points Gained'].shift(i)
    
    team_df['HAvgPts in Last 5 Home Matches'] = column / number_of_previous_matches
    team_df.drop(columns=['Points Gained'], inplace=True)
    
    team_df = team_df[team_df['HomeTeam'] == team]
    temp_df = pd.concat([temp_df, team_df]).reset_index(drop=True)

df = temp_df

C:\Users\choibs\AppData\Local\Temp\ipykernel_10148\1744870593.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row), axis=1)
C:\Users\choibs\AppData\Local\Temp\ipykernel_10148\1744870593.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df['Points Gained'] = team_df.apply(lambda row: getPointsGained(row), axis=1)
C:\Users\choibs\AppData\Local\Temp\ipykernel_10148\1744870593.py:5: SettingWithCopyWarning: 
A value is trying to be s

In [18]:
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HxG,AxG,HPoss,HDaysLastPlayed,ADaysLastPlayed,HxGD in Last 5 Matches,HxGD in Last 5 Home Matches,HAvgPts in Last 5 Matches,HAvgPts in Last 5 Home Matches
0,2017-08-11,arsenal,leicester,4,3,27,6,10,3,9,12,9,4,0,1,0,0,1.53,4.5,6.5,W,2.5,1.5,68.0,5,10,NaN,NaN,NaN,NaN
1,2017-09-09,arsenal,bournemouth,3,0,17,7,9,2,14,10,10,3,0,1,0,0,1.36,5.5,9.0,W,2.2,0.6,58.0,10,10,NaN,NaN,NaN,NaN
2,2017-09-25,arsenal,west-brom,2,0,16,7,6,3,8,17,7,4,1,4,0,0,1.33,5.5,11.0,W,2.2,0.9,69.0,5,5,-1.0,NaN,2.6,NaN
3,2017-10-01,arsenal,brighton,2,0,25,9,12,1,7,8,6,5,0,2,0,0,1.20,6.5,13.0,W,2.4,0.4,64.0,3,7,0.0,NaN,2.6,NaN
4,2017-10-28,arsenal,swansea,2,1,17,4,5,2,9,9,5,2,0,0,0,0,1.25,7.0,12.0,W,2.0,0.9,72.0,4,4,6.0,NaN,2.0,NaN


In [19]:
df.shape

(1900, 30)

## Home Team Win streak

In [20]:
t = 'arsenal'
df[(df['HomeTeam'] == t) | (df['AwayTeam'] == t)].sort_values(by = 'Date').head(20)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Result,HxG,AxG,HPoss,HDaysLastPlayed,ADaysLastPlayed,HxGD in Last 5 Matches,HxGD in Last 5 Home Matches,HAvgPts in Last 5 Matches,HAvgPts in Last 5 Home Matches
0,2017-08-11,arsenal,leicester,4,3,27,6,10,3,9,12,9,4,0,1,0,0,1.53,4.50,6.50,W,2.5,1.5,68.0,5,10,NaN,NaN,NaN,NaN
1216,2017-08-19,stoke,arsenal,1,0,11,18,4,6,6,11,2,9,0,0,0,0,4.50,3.80,1.85,W,0.7,1.5,24.0,7,8,NaN,NaN,NaN,NaN
1122,2017-08-27,liverpool,arsenal,4,0,18,8,10,0,6,9,4,3,2,4,0,0,2.15,3.70,3.50,W,3.1,0.6,48.0,4,8,NaN,NaN,NaN,NaN
1,2017-09-09,arsenal,bournemouth,3,0,17,7,9,2,14,10,10,3,0,1,0,0,1.36,5.50,9.00,W,2.2,0.6,58.0,10,10,NaN,NaN,NaN,NaN
192,2017-09-17,chelsea,arsenal,0,0,13,11,4,2,11,15,5,1,1,3,1,0,1.80,4.00,4.50,D,0.8,1.4,51.0,5,3,NaN,NaN,NaN,NaN
2,2017-09-25,arsenal,west-brom,2,0,16,7,6,3,8,17,7,4,1,4,0,0,1.33,5.50,11.00,W,2.2,0.9,69.0,5,5,-1.0,NaN,2.6,NaN
3,2017-10-01,arsenal,brighton,2,0,25,9,12,1,7,8,6,5,0,2,0,0,1.20,6.50,13.00,W,2.4,0.4,64.0,3,7,0.0,NaN,2.6,NaN
573,2017-10-14,watford,arsenal,2,1,11,9,2,6,8,10,2,5,1,0,0,0,5.25,4.33,1.65,W,1.6,1.0,46.0,10,10,-3.0,NaN,0.4,NaN
384,2017-10-22,everton,arsenal,2,5,8,30,3,14,14,14,0,7,1,1,1,0,3.79,3.60,2.04,L,1.0,3.5,33.0,3,3,-7.0,NaN,1.4,NaN
4,2017-10-28,arsenal,swansea,2,1,17,4,5,2,9,9,5,2,0,0,0,0,1.25,7.00,12.00,W,2.0,0.9,72.0,4,4,6.0,NaN,2.0,NaN
